# Filings Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import csv
from datetime import datetime

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

In [ ]:
%%sql 
update sent_to_gazette
set sent_to_gazette_date = now() 
from (select f.id from  filings f, businesses b, sent_to_gazette stg, businesses_version old, businesses_version new  
          where f.filing_type in ('alteration', 'changeOfName')                  
                and f.business_id=b.id
                and f.id=stg.filing_id
                and f.transaction_id=old.end_transaction_id 
                and f.transaction_id=new.transaction_id                
                and old.legal_name != new.legal_name
                and stg.sent_to_gazette_date is null 
    ) AS subquery
where sent_to_gazette.filing_id=subquery.id; 

In [ ]:
%%sql 
update sent_to_gazette
set sent_to_gazette_date = now() 
from (select f.id from  filings f, businesses b, sent_to_gazette stg  
          where f.filing_type in ('dissolved', 'dissolution')
                and f.business_id=b.id
                and f.id=stg.filing_id
                and stg.sent_to_gazette_date is null
    ) AS subquery
where sent_to_gazette.filing_id=subquery.id; 

In [ ]:
%%sql 
update sent_to_gazette
set sent_to_gazette_date = now() 
from (select f.id from  filings f, businesses b, sent_to_gazette stg    
          where f.filing_type in ('incorporationApplication','amalgamationApplication')      
                and f.business_id=b.id
                and f.id=stg.filing_id
                and stg.sent_to_gazette_date is null
    ) AS subquery
where sent_to_gazette.filing_id=subquery.id; 

In [ ]:
%%sql 
update sent_to_gazette
set sent_to_gazette_date = now() 
from (select f.id from  filings f, businesses b, sent_to_gazette stg    
          where f.filing_type = 'restoration'    
                and f.business_id=b.id
                and f.id=stg.filing_id
                and stg.sent_to_gazette_date is null
    ) AS subquery
where sent_to_gazette.filing_id=subquery.id; 